In [5]:
!pip install keras_tuner


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import pandas as pd
import os

In [7]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive')
else:
    path = r'C:/dev/machine_learning_project/deep_learning/classification/'
    os.chdir(path)

#Importing data

In [8]:
dataframe = pd.read_excel('../../data/chiefs_knife_dataset.xlsx')
index_Ra = dataframe.columns.get_loc('Ra') # index of the surface roughness column for inserting the class. label

lower_specification_limit = 0.125 # lower bound of good quality product region
upper_specification_limit = 0.215  # upper bound of good quality product region

is_between_specification_bounds = (dataframe['Ra']>=lower_specification_limit) & (dataframe['Ra'] < upper_specification_limit)
good_product_range = np.where(is_between_specification_bounds,"good product","poor product")
dataframe.insert(index_Ra +1, 'Quality',good_product_range) # encoding: good quality product := 1 and poor quality product := 0

# constructing Features and Label

In [9]:
X = dataframe.loc[:,'Original_Linienanzahl':'DFT_Median_sobel_Bereich'].values
y = dataframe['Ra'].values

#Splitting dataset into training and test set

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [11]:
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,shuffle=True, random_state=1)

#Scaling the features

In [12]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Model definition

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LeakyReLU, ELU, Activation
from tensorflow.keras.optimizers import Adam, SGD, Nadam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import keras_tuner as kt
from keras_tuner import HyperParameters


hp = kt.HyperParameters()

class DeepRegressionModel:
  def __init__(self):
      """
      initialize the hyperparameters for hyperparameter tuning
      """
      self.neurons_max = 1024
      self.neurons_min = 8
      self.neurons_step = 16
      self.learning_rates = [1e-2, 1e-3, 1e-4,1e-5]
      self.activation_name = ['relu', 'leaky_relu', 'elu', 'selu']
      self.optimizer_name = ['adam', 'nadam','rmsprop','sgd']

  def build_model(self,hp):
      """
      create a NN architecture for the classifiction task
      hp: hyperparameter for hyperparameter tuning
      """
      model = Sequential()

      model.add(Dense(units = hp.Int('hidden_1', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
                      activation =self.set_activation(hp.Choice('activation_1', values=self.activation_name)),input_dim=X_train.shape[1]))

      model.add(Dense(units = hp.Int('hidden_2', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
                      activation = self.set_activation(hp.Choice('activation_2', values=self.activation_name))))

      model.add(Dense(units = hp.Int('hidden_3', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
                      activation = self.set_activation(hp.Choice('activation_3', values=self.activation_name))))

      model.add(Dense(units = hp.Int('hidden_4', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
                      activation = self.set_activation(hp.Choice('activation_4', values=self.activation_name))))

      model.add(Dense(units = hp.Int('hidden_5', min_value=self.neurons_min, max_value=self.neurons_max, step=self.neurons_step),
                      activation = self.set_activation(hp.Choice('activation_5', values=self.activation_name))))

      model.add(Dense(units= 1, activation =self.set_activation(hp.Choice('activation_out', values=self.activation_name))))

      model.compile(optimizer =self.set_optimizer(hp.Choice('optimizer', values=self.optimizer_name), hp.Choice('learning_rate', values=self.learning_rates)) ,
                    loss='mean_squared_error',metrics=[tf.keras.metrics.R2Score(name='r_squared')])
      return model

  def set_activation(self, activation_name):
      """
      set the activation function for the model
      """
      if activation_name == 'leaky_relu':
        activation = LeakyReLU()
      elif activation_name == 'elu':
        activation = ELU()
      elif activation_name == 'selu':
        activation = Activation('selu')
      else:
        activation = Activation(activation_name)
      return activation

  def set_optimizer(self, optimizer_name, learn_rate):
      """
      set the optimizer for the model
      """
      if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learn_rate)
      elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learn_rate)
      elif optimizer_name == 'nadam':
        optimizer = Nadam(learning_rate=learn_rate)
      elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learn_rate)
      return optimizer

  # def r_squared(self,y_true, y_pred):
  #     """
  #     calculate R^2 metric for model evaluation
  #     y_true: true label
  #     y_pred: predicted label
  #     """
  #     y_mean = tf.reduce_mean(y_true)
  #     residual = tf.reduce_sum(tf.square(y_true - y_pred))
  #     total_sum_of_squares  = tf.reduce_sum(tf.square(y_true - y_mean))
  #     r2 = 1 - residual / (total_sum_of_squares)
  #     # r2 = 1 - residual / (total_sum_of_squares + tf.keras.backend.epsilon())
  #     return r2

  # def adjusted_r_squared(self,y_true, y_pred):
  #     """
  #     calculate adjusted R^2 metric for model evaluation
  #     y_true: true label
  #     y_pred: predicted label
  #     """
  #     n = tf.cast(tf.shape(X_train)[0], tf.float32)
  #     p = tf.cast(tf.shape(X_train)[1], tf.float32)

  #     r2 = self.r_squared(y_true, y_pred)
  #     adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
  #     return adjusted_r2

#Random search for hyperparameter tuning

In [14]:
regressor = DeepRegressionModel()

tuner = kt.RandomSearch(
hypermodel=regressor.build_model,
objective=kt.Objective("val_r_squared", direction="max"),
max_trials=50,
executions_per_trial=1,
overwrite=True,
directory="hyperparameter_tuning_regressor",
project_name="knife_regressor",
)

C:\dev\Udemy\Python\py-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=10)
tuner.search(X_train, y_train, batch_size=8, epochs=15, validation_split=0.1)

Trial 99 Complete [00h 02m 16s]
val_r_squared: 0.6067884564399719

Best val_r_squared So Far: 0.6597249507904053
Total elapsed time: 03h 57m 53s

Search: Running Trial #100

Value             |Best Value So Far |Hyperparameter
424               |280               |hidden_1
leaky_relu        |elu               |activation_1
376               |648               |hidden_2
selu              |elu               |activation_2
232               |1000              |hidden_3
relu              |relu              |activation_3
936               |8                 |hidden_4
elu               |selu              |activation_4
136               |616               |hidden_5
leaky_relu        |leaky_relu        |activation_5
selu              |leaky_relu        |activation_out
rmsprop           |rmsprop           |optimizer
0.001             |0.001             |learning_rate

Epoch 1/15
766/766 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.2072 - r_squared: -52.1266 - val_loss: 0.0021 - val_r_squared: 0.4

In [16]:
models = tuner.get_best_models(num_models=8)
tuner.results_summary()

Results summary
Results in hyperparameter_tuning\simple
Showing 10 best trials
Objective(name="val_r_squared", direction="max")

Trial 095 summary
Hyperparameters:
hidden_1: 280
activation_1: elu
hidden_2: 648
activation_2: elu
hidden_3: 1000
activation_3: relu
hidden_4: 8
activation_4: selu
hidden_5: 616
activation_5: leaky_relu
activation_out: leaky_relu
optimizer: rmsprop
learning_rate: 0.001
Score: 0.6597249507904053

Trial 039 summary
Hyperparameters:
hidden_1: 872
activation_1: selu
hidden_2: 40
activation_2: selu
hidden_3: 184
activation_3: leaky_relu
hidden_4: 88
activation_4: leaky_relu
hidden_5: 680
activation_5: elu
activation_out: elu
optimizer: rmsprop
learning_rate: 0.0001
Score: 0.6542302370071411

Trial 047 summary
Hyperparameters:
hidden_1: 280
activation_1: leaky_relu
hidden_2: 376
activation_2: relu
hidden_3: 696
activation_3: elu
hidden_4: 8
activation_4: leaky_relu
hidden_5: 744
activation_5: elu
activation_out: elu
optimizer: rmsprop
learning_rate: 0.001
Score: 0.

C:\dev\Udemy\Python\py-env\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
